# Action Recognition Model
- video 분류 문제 : input 짧은 영상 -> label 분류
- 영상 format은 크게 중요하지 않음 (codec이 있으면 python으로 읽는 건 다 똑같음) 
- UCF dataset
  - https://www.crcv.ucf.edu/data/UCF101.php
  - UCF101, 50, 11 : 숫자는 class의 개수라고 생각하면 됨 ex) 101가지 분류
    - 숫자가 클 수록 task가 어려워짐
  - 제일 작은 dataset 사용해 볼 것임 ([UCF 11](https://www.crcv.ucf.edu/data/UCF_YouTube_Action.php))
    - 29.97 fps : 1초에 총 30여 개의 사진이 있는 것 = 코드로 4초짜리 영상을 읽으면 대략 120개의 사진을 읽어야 하는 것
    - 매우 짧은 영상(4초내외)이고 한 영상에 한 가지 동작만 있음 (걷다가 자전거 타다가 등 하지 않음)

In [1]:
#pip install opencv-python

In [1]:
#pip install tensorflow

In [ ]:
import os
import cv2
from glob import glob

from pprint import pprint
from tqdm import tqdm

import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

import albumentations as A

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '5'  # GPU device number 입력

In [ ]:
file_paths = glob('UCF11_updated_mpg/*/*/*.mpg')
len(file_paths)  # 영상 개수

In [ ]:
print(file_paths[0])
cap = cv2.VideoCapture(file_paths[0])

In [ ]:
frames = []
while True:
    ret, frame = cap.read()  # generator처럼 read를 할 때마다 한 frame 씩 읽음
    if not ret:   # 더 이상 읽을 frame이 없으면 return은 False가 됨
        break

    frame = cv2.resize(frame, (256, 256))
    frame = frame[:, :, [2, 1, 0]]  # 채널 부분만 변경 필요 : BGR -> RGB
    frames.append(frame)

# ★ 반드시 영상 닫아줘야 함
# cap 만드는 순간 이 코드부터 적을 것
cap.release()

잘 읽어 왔는지 확인

In [ ]:
arr = np.array(frames)  # list -> array
plt.figure(figsize=(15, 15))
for i in range(10):
    plt.subplot(10, 3, 1 + 3*i)   # 10*3 = 30개 사진 읽어보고자 함
    plt.imshow(arr[1 + 3*i])
    plt.subplot(10, 3, 2 + 3*i)
    plt.imshow(arr[2 + 3*i])
    plt.subplot(10, 3, 3 + 3*i)
    plt.imshow(arr[3 + 3*i])

In [ ]:
# 방법1
print(len(frames))        # 137개의 frame 읽음
print(len(frames)/29.97)  # 한 영상 당 4초
print('\n')

# 방법2
for file_path in file_paths:
    cap = cv2.VideoCapture(file_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # property frame count
    print(file_path)
    print(length, 'frames')
    print(length / 29.97, 'sec')    
    cap.release()
    break

---
# 1. EDA

In [ ]:
df = pd.DataFrame(
    columns = [
        'file_path',  # 영상 경로
        'frames',     # 몇 frame
        'duration',   # 영상 길이
        'label'       # class
    ]
)

for file_path in file_paths:
    label = file_path.split('/')[1]
    
    cap = cv2.VideoCapture(file_path)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frames/29.97
    
    elem = {
        'file_path': file_path,
        'frames': frames,
        'duration': duration,
        'label': label
    }
    
    df.loc[len(df)] = elem
    cap.release()

In [ ]:
df

### 1.1 같은 label에 속한 영상의 총 길이, 평균 길이

In [ ]:
df_duration_sum_by_label = df.groupby('label').\
    duration.sum().rename('Sum').reset_index().\
    set_index('label')   # label을 index로 사용

df_duration_sum_by_label

In [ ]:
df_duration_avg_by_label = df.groupby('label').\
    duration.mean().rename('Average').reset_index().\
    set_index('label')   # label을 index로 사용

In [ ]:
# index가 같아서 합치기 가능
df_video_stats = \
    pd.concat([df_duration_sum_by_label, df_duration_avg_by_label], axis=1)

df_video_stats

In [ ]:
# y-axis 두 개 사용하기
df_video_stats.plot.bar(secondary_y='Average')

### 1.2 label 내 영상 길이 별 분포

In [ ]:
df['> 10.0 Sec'] = df['duration'] >= 10.0  # 10초 이상의 영상들
df['5.0 - 10.0 Sec'] = \
    (5.0 <= df['duration']) & (df['duration'] < 10.0)
df['2.0 - 5.0 Sec'] = \
    (2.0 <= df['duration']) & (df['duration'] < 5.0)
df['0.0 - 2.0 Sec'] = \
    (0.0 <= df['duration']) & (df['duration'] < 2.0)
df

In [ ]:
df_groupby_label = df.groupby('label')
df_groupby_stats = pd.concat([
    df_groupby_label['0.0 - 2.0 Sec'].
    sum().reset_index().set_index('label'),
    df_groupby_label['2.0 - 5.0 Sec'].
    sum().reset_index().set_index('label'),
    df_groupby_label['5.0 - 10.0 Sec'].
    sum().reset_index().set_index('label'),
    df_groupby_label['> 10.0 Sec'].          # False = 0, True = 1
    sum().reset_index().set_index('label'),  # 결국 label 별 개수 세는 것
], axis=1)

df_groupby_stats

In [ ]:
df_groupby_stats.plot.bar(stacked=True, ylabel='Number of Videos')

---
# 2. Data Loader

- CNN (2D) 모델로 video classification 하기 위한 데이터 처리 방법
- video는 frame의 연속이기 때문에 각 frame을 tree classification model로 분류하고, 그 frame들의 평균 값을 내면 video 예측 가능
  - 이를 위해서는 video에서 각 frame을 모두 분리해서 저장 필요

In [ ]:
label_dir = glob('UCF11_updated_mpg/*')
label_dir

현재 상황
- 11개의 class가 있음
- class 당 25개의 영상 group이 있음
  - 한 group 안에서 train, validation을 나누면, 결국 비슷한 영상이기 때문에 validation이 높게 측정될 수 있음
  - 이러한 현상을 방지하기 위해 group을 미리 분류하고자 함
    - 1\~20 : training, 20\~25 : validation

### 2.1 train, validation split

In [ ]:
train_df = pd.DataFrame(
    columns=['file_path', 'label']
)


valid_df = pd.DataFrame(
    columns=['file_path', 'label']
)

label_dirs = glob('UCF11_updated_mpg/*')

for label_dir in label_dirs:
    file_dir = glob(label_dir + '/v_*')
    random.shuffle(file_dir)   # inplace
    
    for i in range(20):
        train_dir = file_dir[i]
        label = train_dir.split('/')[-1].split('_')[1]
        
        # 학습에 사용할 video를 random으로 하나 가져오기
        file_path = random.choice(
            glob(train_dir + '/*')
        )
        
        train_df.loc[len(train_df)] = [file_path, label] # 마지막 행으로 추가
        
    for i in range(20, 25):
        valid_dir = file_dir[i]
        label = valid_dir.split('/')[-1].split('_')[1]
        
        # 학습에 사용할 video를 random으로 하나 가져오기
        file_path = random.choice(
            glob(valid_dir + '/*')
        )
        
        valid_df.loc[len(valid_df)] = [file_path, label]

In [ ]:
print(len(train_df), len(valid_df))

In [ ]:
train_df

### 2.2 영상 frame에 label 붙여서 저장하기

image 저장 directory 만들기

In [ ]:
os.mkdir('UCF11_update_png')
os.mkdir('UCF11_update_png/train')
os.mkdir('UCF11_update_png/valid')

train_df.to_csv('UCF11_train_video.csv', index=False)
valid_df.to_csv('UCF11_valid_video.csv', index=False)

한 영상에서 제일 앞 10개의 frame으로 image training

In [ ]:
max_frame = 10
SAVE_DIR = 'UCF11_update_png/'

for i, elem in train_df.iterrows():
    cap = cv2.VideoCapture(elem['file_path'])
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (256, 256))
        frames.append(frame)
        
        if len(frames) == max_frame:
            break
        
    label = elem['label']
    
    for j, frame in enumerate(frames):
        file_name = f'train/{label}_{i}_{j}.png'
        cv2.imwrite(SAVE_DIR + file_name, frame)
    
    cap.release

In [ ]:
print(len(glob(SAVE_DIR + 'train/*')))  # 2200개 저장됨

In [ ]:
max_frame = 10
SAVE_DIR = 'UCF11_update_png/'

for i, elem in valid_df.iterrows():
    cap = cv2.VideoCapture(elem['file_path'])
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (256, 256))
        frames.append(frame)
        
        if len(frames) == max_frame:
            break
        
    label = elem['label']
    
    for j, frame in enumerate(frames):
        file_name = f'valid/{label}_{i}_{j}.png'
        cv2.imwrite(SAVE_DIR + file_name, frame)
    
    cap.release

In [ ]:
print(len(glob(SAVE_DIR + 'valid/*')))  # 550개 저장됨

---
# 3. Model training - CNN approach

- 11개의 class가 있으니 multi-class classification

In [ ]:
# multi-class를 위해 label에 고유의 int 값 부여 
LABEL_INT_DICT = np.unique(pd.read_csv('UCF11_train_video.csv')['label'])
LABEL_STR_DICT = {k:v for v, k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_INT_DICT)
pprint(LABEL_STR_DICT)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, image_size,
                mode='train', shuffle=True):
        assert mode in ['train', 'valid']
        
        self.mode = mode
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.image_size = image_size
        
        if self.mode == 'train':
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=15
                )
            ])
        
        self.img_paths = glob(
            f'UCF_updated_png/{mode}/*.png'        
        )
        
    def __len__(self):
        return math.ceil(len(self.img_paths) / self.batch_size)  # epoch 당 step 개수
    
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.img_paths[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
    
    def get_data(self, data): # 이미지 읽기
        batch_x = []
        batch_y = []
        
        for img_path in data:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (self.image_size, self.image_size))
            
            if self.mode == 'train':
                augmented = self.transform(image=img)
                img = augmented['image']
                
            img = img.astype('float32')
            img = img / 255.
            
            label = img_path.split('/')[-1].split('_')[0]
            
            # sparse를 사용하니 따로 one-hot encoding은 필요없음
            label = LABEL_STR_DICT[label]
            
            batch_x.append(img)
            batch_y.append(label)
        
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.img_paths)

train_generator = DataGenerator(
    mode='train',
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator = DataGenerator(
    mode='valid',
    batch_size=128,
    image_size=256,
    shuffle=True
)

In [ ]:
# 이미지와 label이 잘 matching 되어 구현되었는지 확인
for batch in train_generator:
    X, y = batch
    
    plt.figure(figsize=(15, 15))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(X[i])
        plt.title(LABEL_INT_DICT(y[i]))
        plt.axis('off')
    break

In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer(
        'http://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2', # model import
        trainable=True  # finetuning 하기 위함
    ),
    keras.layers.Dense(len(LABEL_INT_DICT), activation='softmax') # 11개 class
])

model.build([None, 256, 256, 3]) # input size
adam = keras.optimizers.Adam(lr=0.0001)
model.compile(
    optimizer=adam,
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)

model.summary()

- weight 파일 저장

In [ ]:
os.mkdir('UCF11_weights')

filepath = 'UCF11_weights/{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(  # model 구조 전체 저장 (weight 포함)
    filepath,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=False # 성능이 update
)

history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1,
    callbacks=[model_checkpoint]
)

- data가 적다보니 10 epoch에서 벌써 overfitting 발생
- 조금 더 가벼운 model 사용해도 괜찮음

In [ ]:
history = history.history

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_acc'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy')
plt.show()

- 위 모델은 이미지 하나하나에 대한 분류 성능
- 동영상 하나를 넣었을 때 결과가 나오는 모델이 아님
- 각각의 frame에 대한 예측값을 가져와서 평균을 내는 작업 필요

In [ ]:
csv_path = 'UCF_valid_video.csv'
valid_df = pd.read_csv(csv_path)  # 동영상 Link

# 예측을 잘 했는지
correct = 0

for i, elem in tqdm(valid_df.iterrows(), total=len(valid_df)):
    cap = cv2.VideoCapture(
        elem['file_path']
    )
    
    preds = []
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (256, 256))
        frame = frame.astype('float32')
        frame /= 255.
        
        pred = model.predict(frame[np.newaxis, ...])
        preds.append(pred)
    
    preds = np.array(preds).mean(axis=0) # 각 column 평균 계산
    
    label = LABEL_INT_DICT[np.argmax(preds)]    
    if label == elem['label']:
        correct += 1
    
    cap.release()

# 동영상 predictio에 대한 accuracy
# 지금은 single frame에 대한 성능과 크게 차이 나지는 않음
print('Accuracy: ', correct / len(valid_df))

---
# 4. 예측 결과 시각화 및 문제 해결방법

### 4.1 시각화

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
model = keras.models.load_model(
    'UCF11_weights/09-0.87.hdf5',
    # keras 제공 외 layer도 사용해서 model 구현했기 때문에 아래 코드 필요
    custom_objects={'KerasLayer': hub.KerasLayer}
)

model.build([None, 256, 256, 3])
model.summary()

- 방법
  - 21번째 영상에 대해 각 frame에 대한 prediction 생성
  - prediction을 frame에 덮어 씌워서 다시 동영상으로 변환
  - 'ori.avi' 파일로 저장
  - 해당 영상 재생해 보고 예측 잘 됐는지 확인
- 영상 보면 상단 prediction 결과 글씨가 잠깐 바뀌는 error 발생 이유
  - frame 한 장 한 장 별로 예측하고, 이전과 이후 상황을 고려하지 않기 때문에, 해당 frame에서 예측 값이 틀린 것
  - 이걸 flickering effect 라고 함
  - ★ 해결방법 : rolling average 사용하기
- rolling average
  - 한 frame을 예측할 때 이전 frame들에 대한 예측 값도 모두 더해서 평균을 내는 것
  - flickering effect를 막아줌

In [ ]:
csv_path = 'UCF11_valid_video.csv'
valid_df = pd.read_csv(csv_path)

# 21번째 영상 사용
idx = 21
elem = valid_df.iloc[idx]

cap = cv2.VideoCapture(elem['file_path'])

# fourcc : encoder를 정하는 것
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# writer = cv2.VideoWriter('ori.avi', fourcc, 30, (256, 256), True) # 파일명, encoding, fps, output image
writer = cv2.VideoWriter('roll.avi', fourcc, 30, (256, 256), True) # rolling average 사용

queue = [] ### 이전까지 prediction을 모두 가지는 list
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 기존 frame은 유지하고 모델을 위한 전처리 frame_ 생성
    # 모델에 필요한 image와 output image가 다르기 때문 (기존 frame 필요)
    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ /= 255.

    pred = model.predict(frame_[np.newaxis, ...])
    # label = LABEL_INT_DICT[np.argmax(preds)]

    ### rolling average
    queue.append(pred)
    results = np.array(queue).mean(axis=0)    
    label = LABEL_INT_DICT[np.argmax(results)]
    
    # frame에 label 덮어쓰기
    frame = cv2.resize(frame, (256, 256))
    cv2.putText(
        frame,
        label,
        (50, 45),  # label이 덮어 씌워질 좌표
        cv2.FONT_HERSHEY_SIMPLEX,  # font
        1.5,  # font size
        (0, 255, 0),  # font color
        5  # font 굵기
    )
    writer.write(frame)    

cap.relaese()
writer.release()  # writer도 반드시 release 필요

---
# 5. Data preparation - RNN approach

- 동영상은 frame의 연속인만큼 frame 순서에도 의미가 있는데 위와 같이 예측하면 그게 반영이 안됨
- RNN
  - frame 간 관계 학습, 동영상의 특징이 모델에 더 잘 반영됨
  - 보통 CNN-RNN 모델로 많이 사용함
    - CNN : 한 동영상이 들어오면 각 frame에 대한 feature 추출
    - CNN에서 추출된 feature를 RNN에 input
    - 이때 feature를 미리 따로 저장해 놓으면, RNN 모델을 학습할 때 매번 CNN 모델을 사용해서 feature를 extract 하지 않아도 되니 학습 속도가 매우 빨라짐

### 5.1 feature extract & save - CNN approach

In [ ]:
model = keras.model.load_model(
    'UCF11_weights/10-0.87.hdf5',
    custom_objects={'KerasLayer':hub.KerasLayer}
)

In [ ]:
model.summary()

In [ ]:
feature_extractor = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 3)), # input size
        model.layers[0] # 첫 번째 layer 결과값 가져오기 (dense layer는 사용 X) : 1280
    ],
    name = 'feature_extractor'
)

feature_extractor.summary()

In [ ]:
os.mkdir('UCF11_updated_npy')
os.mkdir('UCF11_updated_npy/train')
os.mkdir('UCF11_updated_npy/valid')

- 10개의 frame만 사용해서 학습하고자 함

In [ ]:
max_frames = 10
SAVE_DIR = 'UCF11_updated_npy/'
train_df = pd.read_csv('UCF11_train_video.csv')
valid_df = pd.read_csv('UCF11_valid_video.csv')

- train set

In [ ]:
for i, elem in tqdm(train_df.iterrows(), total=len(train_df)):
    
    label = elem['label']
    
    cap = cv2.VideoCapture(
        elem['file_path']
    )
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ /= 255.
    
    frames.append(frame_) # 앞의 10개의 frame을 순서대로 append
    if len(frames) == max_frames:
        break
        
    cap.release()
    
    # frame 하나 씩을 예측하려면 시간이 많이 걸림
    # frame을 하나의 batch로 만들어서 한 번에 예측하면 속도 훨씬 빨라짐
    # 순서대로 append 했으니 별도 코딩 없이 그대로 RNN에 넣어주면 됨
    frames = np.array(frames)   # batch
    features = feature_extractor.predict(frames)  # 한 번에 예측
    
    file_name = SAVE_DIR + f'train/{label}_{i}.npy'
    np.save(file_name, features)

- validation set

In [ ]:
for i, elem in tqdm(valid_df.iterrows(), total=len(valid_df)):
    
    label = elem['label']
    
    cap = cv2.VideoCapture(
        elem['file_path']
    )
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ /= 255.
    
    frames.append(frame_) # 앞의 10개의 frame을 순서대로 append
    if len(frames) == max_frames:
        break
        
    cap.release()
    
    # frame 하나 씩을 예측하려면 시간이 많이 걸림
    # frame을 하나의 batch로 만들어서 한 번에 예측하면 속도 훨씬 빨라짐
    # 순서대로 append 했으니 별도 코딩 없이 그대로 RNN에 넣어주면 됨
    frames = np.array(frames)   # batch
    features = feature_extractor.predict(frames)  # 한 번에 예측
    
    file_name = SAVE_DIR + f'valid/{label}_{i}.npy'
    np.save(file_name, features)

- 제대로 됐는지 개수 확인

In [ ]:
print(len(glob(SAVE_DIR + 'train/*''))) # 220개
print(len(glob(SAVE_DIR + 'train/*''))) # 55개

### 5.2 간단한 RNN classification model 만들어 보기

In [ ]:
MAX_FEATURE = 10 # 연속된 이미지가 들어오는데 총 몇 개의 이미지(feature)가 한 번에 들어오는지
NUM_FEATURES = 1280 # 위에서 CNN 모델 결과 1280개 feature 있었음

def build_model():
    inputs = keras.Input((MAX_FRAMES, NUM_FEATURES))
    # 64 : node 개수
    # return_sequences=False
    #  - 10개(max_feature)가 아니라 1개 값만 나옴
    #  - LSTM layer를 쌓을 수 없게 됨
    x = keras.layers.LSTM(64, return_sequences=True)(inputs)
    x = keras.layers.LSTM(64, return_sequences=False)(x)
    x = keras.layers.Dropout(0.3)(x)
    outputs = keras.layers.Dense(len(LABEL_INT_DICT), activation='softmax')(x) # 11개 class 예측
    
    model = keras.Model(inputs, outputs)
    return model

adam = keras.optimizers.Adam(lr=0.0001)
model = build_model()

model.compile(
    optimizer=adam,
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)

model.summary()

### 5.3 Data Generator 생성

- image를 load 하는 게 아니라 numpy로 된 feature를 load 하는 거라 코드 일부 수정 필요
  - image augmentation 사용 불가 -> 제외

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size,
                mode='train', shuffle=True):
        assert mode in ['train', 'valid']
        
        self.mode = mode
        self.shuffle = shuffle
        self.batch_size = batch_size
        
        self.npy_paths = glob(
            f'UCF_updated_npy/{mode}/*.npy'        
        )
        
    def __len__(self):
        return math.ceil(len(self.npy_paths) / self.batch_size)  # epoch 당 step 개수
    
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.npy_paths[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
    
    def get_data(self, data): # 이미지 읽기
        batch_x = []
        batch_y = []
        
        for npy_path in data:
            
            # CNN을 거쳐 모두 전처리를 했기 때문에 RNN은 매우 간단하게 구현 가능
            npy = np.load(npy_path)
            
            label = npy_path.split('/')[-1].split('_')[0]
            
            # sparse를 사용하니 따로 one-hot encoding은 필요없음
            label = LABEL_STR_DICT[label]
            
            batch_x.append(npy)
            batch_y.append(label)
        
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.npy_paths)

train_generator = DataGenerator(
    mode='train',
    batch_size=128,
    shuffle=True
)

valid_generator = DataGenerator(
    mode='valid',
    batch_size=128,
    shuffle=True
)

In [ ]:
for x, y in train_generator:
    print(x.shape, y.shape)
    break

RNN 모델이 가벼워서 금방 돔

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    verbose=1,
)

### 결과
- CNN보다 성능이 훨씬 올라감
- task가 어려울수록 CNN과 함께 RNN 사용 필수

### 모델 성능을 높이려면?
- 지금 우리는 downsampling을 했었는데 더 많은 sample 사용하기
- 10 frame 보다 더 많은 frame 사용하기

In [ ]:
history = history.history

plt.figure(figsize=(10, 5))
plt.subplot(1,2,1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss')

plt.subplot(1,2,2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_acc'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy')
plt.show()